# School Shooting Tables — Initial Exploration

Pull core tables from Supabase and run a first-pass structure and completeness review.

## Environment setup
Load credentials from `.env`, initialize the Supabase client, and import analysis helpers.

In [ ]:
import os
import pandas as pd
from supabase import create_client
from dotenv import load_dotenv

load_dotenv(override=True)

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

: 

## Data retrieval
Reusable helper to pull a full table with simple pagination to avoid size limits.

In [ ]:
def fetch_table(table_name, page_size=2000):
    start = 0
    rows = []
    while True:
        resp = (
            supabase
            .table(table_name)
            .select("*")
            .order("id")
            .range(start, start + page_size - 1)
            .execute()
        )
        batch = resp.data
        if not batch:
            break
        rows.extend(batch)
        start += page_size
    return pd.DataFrame(rows)

## Pull core tables
Incident, victim, shooter, and weapon tables are loaded into data frames.

In [ ]:
tables = ["incident", "victim", "shooter", "weapon"]
data = {name: fetch_table(name) for name in tables}

# Row counts as a quick completeness check
{name: df.shape[0] for name, df in data.items()}

## Column overview
List columns for each table to understand available fields.

In [ ]:
{name: df.columns.tolist() for name, df in data.items()}

## Missing values
Count nulls per column to spot data quality issues.

In [ ]:
null_counts = {name: df.isna().sum().sort_values(ascending=False) for name, df in data.items()}
null_counts

## Missing value rates
Null counts as percentages for easier comparison across tables.

In [ ]:
null_rates = {name: (df.isna().mean().sort_values(ascending=False) * 100).round(2) for name, df in data.items()}
null_rates

## Sample records
A quick glance at the first few rows of each table.

In [ ]:
{name: df.head() for name, df in data.items()}